In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ML Tuning").getOrCreate()

spark

In [20]:
import os
pwd = os.getcwd()
print(pwd)
print("{}/data/miserables_full.txt".format(pwd))

/home/jayl/datascientestfile/spark
/home/jayl/datascientestfile/spark/data/miserables_full.txt


In [21]:
df_full =spark.read.csv("{}/data/YearPredictionMSD.txt".format(pwd),header=False)
df_full.show(5)

+----+--------+--------+--------+--------+---------+---------+---------+---------+--------+--------+-------+--------+--------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------+----------+----------+---------+----------+---------+---------+---------+---------+---------+--------+--------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+----------+---------+---------+---------+---------+---------+--------+---------+---------+----------+---------+----------+---------+---------+---------+---------+---------+----------+----------+---------+---------+--------+---------+---------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------+----------+---------+---------+---------+---------+----------+--------+---------+--------+
| _c0|     _c1|     _c2|     _c3|     _c4|      _c5|      _c6|      _c7|      _c8|     _c9|    

In [22]:
#获得10%的数据，并只取前13列
# 这是我第一反应取得的
df_full = df_full.select(df_full.columns[:14])
df_full.show(5)

+----+--------+--------+--------+--------+---------+---------+---------+---------+--------+--------+-------+--------+--------+
| _c0|     _c1|     _c2|     _c3|     _c4|      _c5|      _c6|      _c7|      _c8|     _c9|    _c10|   _c11|    _c12|    _c13|
+----+--------+--------+--------+--------+---------+---------+---------+---------+--------+--------+-------+--------+--------+
|2001|49.94357|21.47114|73.07750| 8.74861|-17.40628|-13.09905|-25.01202|-12.23257| 7.83089|-2.46783|3.32136|-2.31521|10.20556|
|2001|48.73215|18.42930|70.32679|12.94636|-10.32437|-24.83777|  8.76630| -0.92019|18.76548| 4.59210|2.21920| 0.34006|44.38997|
|2001|50.95714|31.85602|55.81851|13.41693| -6.57898|-18.54940| -3.27872| -2.35035|16.07017| 1.39518|2.73553| 0.82804| 7.46586|
|2001|48.24750|-1.89837|36.29772| 2.58776|  0.97170|-26.21683|  5.05097|-10.34124| 3.55005|-6.36304|6.63016|-3.35142|37.64085|
|2001|50.97020|42.20998|67.09964| 8.46791|-15.85279|-16.81409|-12.48207| -9.37636|12.63699| 0.93609|1.60923| 2.

In [30]:
# 可以不用col实现
from pyspark.sql.functions import col
exprs = [col(c).cast("double") for c in df_full.columns[1:13]]
exprs
df = df_full.select(df_full._c0.cast("int"),*exprs)
df.show(5)

df = df.sample(False,0.1,seed =111)

df.sample(False,0.001,seed =222).toPandas().head(5)

+----+--------+--------+--------+--------+---------+---------+---------+---------+--------+--------+-------+--------+
| _c0|     _c1|     _c2|     _c3|     _c4|      _c5|      _c6|      _c7|      _c8|     _c9|    _c10|   _c11|    _c12|
+----+--------+--------+--------+--------+---------+---------+---------+---------+--------+--------+-------+--------+
|2001|49.94357|21.47114| 73.0775| 8.74861|-17.40628|-13.09905|-25.01202|-12.23257| 7.83089|-2.46783|3.32136|-2.31521|
|2001|48.73215| 18.4293|70.32679|12.94636|-10.32437|-24.83777|   8.7663| -0.92019|18.76548|  4.5921| 2.2192| 0.34006|
|2001|50.95714|31.85602|55.81851|13.41693| -6.57898| -18.5494| -3.27872| -2.35035|16.07017| 1.39518|2.73553| 0.82804|
|2001| 48.2475|-1.89837|36.29772| 2.58776|   0.9717|-26.21683|  5.05097|-10.34124| 3.55005|-6.36304|6.63016|-3.35142|
|2001| 50.9702|42.20998|67.09964| 8.46791|-15.85279|-16.81409|-12.48207| -9.37636|12.63699| 0.93609|1.60923| 2.19223|
+----+--------+--------+--------+--------+---------+----

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12
0,1994,30.16898,11.09407,-43.28152,-5.33756,-18.16409,-11.91031,10.17781,-10.63273,29.40632,9.45716,-3.83821,6.23310
1,2008,34.74711,-71.79470,14.96358,-21.79531,-23.05203,-30.13815,-2.81365,1.39191,15.86665,5.78299,-0.07879,0.59716
2,1997,30.20292,-163.91898,28.74562,-21.31066,18.44927,-19.16093,-11.19296,3.80094,-23.12284,-8.21416,-0.86751,-0.15789
3,2008,44.85505,-18.75080,-16.49408,-17.80846,4.73053,-5.21701,-10.13801,-17.42086,10.27341,-1.76861,-0.06202,13.85828
4,1998,39.84637,-58.22576,48.75672,1.68587,3.44955,19.54420,-30.79726,6.33024,-6.77604,-1.58842,0.09150,14.04782


In [38]:
# (b) Convertir cette base de données au format svmlib dans une variable df_ml.
# help(spark.createDataFrame)
from pyspark.ml.linalg import DenseVector

df_ml = df.rdd.map(lambda x: (x[0],DenseVector(x[1:13])))
df_ml = spark.createDataFrame(df_ml,["label","features"])

df_ml.show(5)
# (c) Afficher un extrait de cette base de données.
df_ml.sample(False,0.001,seed=666).toPandas().head(5)




+-----+--------------------+
|label|            features|
+-----+--------------------+
| 2001|[49.94357,21.4711...|
| 2002|[37.66498,-34.059...|
| 2004|[26.51957,-148.15...|
| 1999|[39.11695,-8.2976...|
| 1997|[37.45034,11.4261...|
+-----+--------------------+
only showing top 5 rows



,label,features
0,2005,"[41.61498, -62.4382, -5.51067, -23.1615, -5.55..."
1,1962,"[40.66192, 3.59829, 28.08621, 42.16801, -9.905..."
2,1998,"[51.44046, 67.68488, 29.81125, 2.53634, -22.47..."
3,1995,"[44.85855, -7.55444, 41.4515, -5.15161, -22.16..."
4,2009,"[45.77231, 3.13134, -25.74867, -1.85734, -9.82..."


In [39]:
# (d) Séparer df_ml en un ensemble train et un ensemble test comprenant respectivement 80% et 20% des données.
(train,test)=df_ml.randomSplit([0.8,0.2],seed=666)


In [41]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="label",featuresCol="features")

In [43]:
# 2. Création d'une grille de paramètres
# (a) Créer une grille de paramètres, appelée param_grid, 
# contenant les valeurs 0, 0.5 et 1 pour les paramètres regParam et elasticNetParam.
from pyspark.ml.tuning import ParamGridBuilder

param_grid = ParamGridBuilder().\
            addGrid(lr.regParam,[0,0.5,1]).\
            addGrid(lr.elasticNetParam,[0,0.5,1]).\
            build()




In [50]:
# (a) Importer la fonction RegressionEvaluator du package pyspark.ml.evaluation.
# (b) Créer un évaluateur evaluator prenant en compte la métrique d'évaluation R² r2.
 
from pyspark.ml.evaluation import RegressionEvaluator
ev = RegressionEvaluator(predictionCol="prediction",
                         labelCol="label",
                         metricName="r2"
                         )



In [54]:
# (a) Importer la fonction CrossValidator du package pyspark.ml.tuning.
# (b) Créer un objet CrossValidator à 3 folds nommé cv prenant comme argument l'estimateur lr, 
# la grille de paramètres et l'évaluateur déjà construits.


from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=param_grid,
                    evaluator=ev,
                    numFolds=3
)

In [55]:
# (a) Dans une variable cv_model, appliquer le crossValidator à la base de données train,
#  à l'aide de la méthode fit.
from time import time 
t0 = time()
cv_model =cv.fit(train)
t_delta = time()-t0
print(f"execute time use {t_delta}")

24/01/08 02:13:45 WARN Instrumentation: [2dd0df38] regParam is zero, which might cause numerical instability and overfitting.
24/01/08 02:13:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/08 02:13:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
24/01/08 02:13:51 WARN Instrumentation: [cb426fc9] regParam is zero, which might cause numerical instability and overfitting.
24/01/08 02:13:52 WARN Instrumentation: [0a23e215] regParam is zero, which might cause numerical instability and overfitting.
24/01/08 02:14:02 WARN Instrumentation: [5cf2e2f5] regParam is zero, which might cause numerical instability and overfitting.
24/01/08 02:14:05 WARN Instrumentation: [7216cc7e] regParam is zero, which might cause numerical instability and overfitting.
24/01/08 02:14:06 WARN Instrumentation: [175010b3] regParam is zero, which might cause numerical instability and overfitting.
24/01/08 02:14:12 WARN Ins

execute time use 46.304951906204224


In [58]:
# (c) Utiliser ce modèle pred_train = cv_model.transform(train)
pred_test = cv_model.transform(test)pour prédire l'année de sortie des chansons de l'échantillon train dans un DataFrame pred_train.
# (d) Utiliser ce modèle pour prédire l'année de sortie des chansons de l'échantillon test dans un DataFrame pred_test.



In [61]:
# (e) Calculer le RMSE obtenu sur l'échantillon test.

rmse_train = ev.setMetricName("rmse").evaluate(pred_train)
rmse_test = ev.setMetricName("rmse").evaluate(pred_test)

print(f'rmse train is {rmse_train}, rmse test is {rmse_test}')

rmse train is 10.03126479176896, rmse test is 10.019568855619077


In [63]:
# (a) Afficher les coefficients obtenus par le modèle optimal.
cv_model.bestModel.coefficients


DenseVector([0.7434, -0.0552, -0.0701, 0.1436, 0.0147, -0.2166, -0.0557, -0.0618, -0.1174, 0.0842, -0.4503, 0.0428])